In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%capture
!pip install flair
!pip install scispacy==0.2.5
# Restart runtime after installing
!pip install --target=/content/data https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.2.5/en_core_sci_sm-0.2.5.tar.gz
# New version of huggingface transformers 4.0.0 breaks flair transformer embeddings
#!pip install transformers==3.5.1

In [ ]:
import pandas as pd
import unicodedata

all_posts = pd.read_csv('/content/drive/My Drive/CSE573_SemanticWebMining/scraped_data.tsv', sep='\t', encoding='utf-8')
#Remove \xa0, \n, \r, \t
all_posts['text'] = [unicodedata.normalize("NFKD", text).replace('\n', ' ').replace('\r', ' ').replace('\t', ' ') for text in all_posts['text']]
all_posts

,title,url,post_num,text
0,Feeling Abused by Expectations on LTC Managers...,https://covid19.camhx.ca/mod/forum/discuss.php...,0,I am becoming more and more frustrated by the...
1,Feeling Abused by Expectations on LTC Managers...,https://covid19.camhx.ca/mod/forum/discuss.php...,1,"Hi Mustang Sally, thank you for sharing your ..."
2,"Is this ""Breathe Easy"" Mask Enhancer safe?",https://covid19.camhx.ca/mod/forum/discuss.php...,0,"Hello, I saw this new apparatus and wondering..."
3,"Is this ""Breathe Easy"" Mask Enhancer safe?",https://covid19.camhx.ca/mod/forum/discuss.php...,1,"Hi there, thank you for your post! We are hap..."
4,"Is this ""Breathe Easy"" Mask Enhancer safe?",https://covid19.camhx.ca/mod/forum/discuss.php...,2,Thank you for the response but that further c...
...,...,...,...,...
180,Supports for Parents/Caregivers,https://covid19.camhx.ca/mod/forum/discuss.php...,0,Just thought I'd share an excellent FREE reso...
181,Supports for Parents/Caregivers,https://covid19.camhx.ca/mod/forum/discuss.php...,1,"Thanks for this post boymom! Yes, it would be..."
182,Supports for Parents/Caregivers,https://covid19.camhx.ca/mod/forum/discuss.php...,2,You're welcome! Happy to help any way I can :...
183,Pregnant and worried,https://covid19.camhx.ca/mod/forum/discuss.php...,0,I am 8 months pre...


In [ ]:
from flair.data import Sentence
from flair.models import MultiTagger
from flair.tokenization import SciSpacyTokenizer
from tqdm.notebook import tqdm

# make a sentence and tokenize with SciSpaCy
print('Tokenizing...')
sentences = []
for text in tqdm(all_posts['text']):
    sentences.append(Sentence(text, use_tokenizer=SciSpacyTokenizer()))

# load biomedical tagger
tagger = MultiTagger.load("hunflair")

# tag sentence
print('Predicting...')
for sent in tqdm(sentences):
    tagger.predict(sent)

Tokenizing...



2021-02-22 11:22:18,431 loading file /root/.flair/models/hunflair-celline-v1.0.pt
2021-02-22 11:22:36,015 loading file /root/.flair/models/hunflair-chemical-full-v1.0.pt
2021-02-22 11:22:54,063 loading file /root/.flair/models/hunflair-disease-full-v1.0.pt
2021-02-22 11:23:12,540 loading file /root/.flair/models/hunflair-gene-full-v1.0.pt
2021-02-22 11:23:29,418 loading file /root/.flair/models/hunflair-species-full-v1.1.pt
Predicting...


In [ ]:
all_posts.iloc[5]['text']

' Hi there, while wearing a mask that fits closely over your nose, cheeks and chin helps to avoid air from coming through the edges of the mask, some people may feel that it is harder to breathe. This trade-off can make it difficult for some people, making it harder for them to keep their masks on for extended periods of time.   Again, though we cannot comment on the safety for any commercial mask/accessories worn either underneath or in front of the mask, I do want to kindly note the following disclaimer that is mentioned on the website: The Cool TurtleTM product is intended for use with non-medical masks only. Do not use for any medical purposes. For guidance on the proper use of non-medical masks, please feel free to review the Non-Medical Masks and Face Covering page on the Government of Canada website. My apologies that we cannot give you a direct answer to your question. We hope these resources can help you make an informed decision regarding your safety.  '

In [ ]:
cnt = 0
all_ents = []
for sent in tqdm(sentences):
    text_ents = []
    for entity in sent.get_spans():
        text_ents.append(str(entity))
    if len(text_ents):
        all_ents.append(text_ents)
    else:
        all_ents.append('-')

all_posts['entities'] = all_ents
print(all_posts.iloc[0])
all_posts.to_csv('/content/drive/MyDrive/CSE573_SemanticWebMining/HUNFLAIR_predictions.tsv', sep='\t', index=False)


title       Feeling Abused by Expectations on LTC Managers...
url         https://covid19.camhx.ca/mod/forum/discuss.php...
post_num                                                    0
text         I am becoming more and more frustrated by the...
entities                                                    -
Name: 0, dtype: object
